In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
203,ABW,North America,Aruba,2020-10-07,4094.0,15.0,22.857,31.0,1.0,0.714,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
485,AFG,Asia,Afghanistan,2020-10-07,39548.0,62.0,42.000,1469.0,2.0,1.571,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
685,AGO,Africa,Angola,2020-10-07,5725.0,355.0,117.143,211.0,18.0,4.571,...,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581


In [2]:
## dump the file for using it in other nb
c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"data\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_deaths_predict
0,2020-10-06,France,636955.0,32340.0
0,2020-10-06,China,90675.0,4744.0
0,2020-10-06,Italy,329755.0,36034.0
0,2020-10-06,Spain,822383.0,32245.0
0,2020-10-06,United States,7537548.0,211288.0
0,2020-10-06,World,36122289.0,1052366.0
0,2020-10-06,United Kingdom,535248.0,42425.0
0,2020-10-06,Germany,304495.0,9555.0
0,2020-10-06,Iran,478711.0,27385.0
0,2020-10-06,Turkey,327166.0,8546.0


In [4]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-07,France,636955.0,634763.0,32340.0,32365.0,2192.0,-25.0
0,2020-10-07,China,90675.0,90667.0,4744.0,4739.0,8.0,5.0
0,2020-10-07,Italy,329755.0,330263.0,36034.0,36030.0,-508.0,4.0
0,2020-10-06,Spain,822383.0,825410.0,32245.0,32486.0,-3027.0,-241.0
0,2020-10-07,United States,7537548.0,7501612.0,211288.0,210909.0,35936.0,379.0
0,2020-10-07,World,36122289.0,35848254.0,1052366.0,1048181.0,274035.0,4185.0
0,2020-10-07,United Kingdom,535248.0,530113.0,42425.0,42445.0,5135.0,-20.0
0,2020-10-07,Germany,304495.0,306086.0,9555.0,9562.0,-1591.0,-7.0
0,2020-10-07,Iran,478711.0,479825.0,27385.0,27419.0,-1114.0,-34.0
0,2020-10-07,Turkey,327166.0,327557.0,8546.0,8553.0,-391.0,-7.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")